# R_V Toolkit Tutorial

Welcome to the R_V Toolkit! This tutorial will guide you through measuring geometric signatures of recursive self-reference in transformer models.

## What You'll Learn
1. Computing R_V (Representational Volume) for transformer activations
2. Running activation patching experiments
3. Analyzing results with statistical tests

## Prerequisites
- Python 3.10+
- PyTorch
- A transformer model (we'll use a small one for demos)

In [ ]:
# Install the toolkit
!pip install -e .

## Part 1: Understanding R_V

R_V (Representational Volume) measures the effective dimensionality of value activations in transformers using the **participation ratio**:

$$R_V = \frac{(\sum_i \sigma_i^2)^2}{\sum_i \sigma_i^4}$$

- **High R_V**: Information distributed across many dimensions (baseline processing)
- **Low R_V**: Information compressed into fewer dimensions (recursive self-reference)

In [ ]:
import torch
import numpy as np
from rv_toolkit import compute_rv, compute_participation_ratio

# Example 1: Simulate baseline activations (high dimensional)
baseline_v = torch.randn(16, 4096)  # 16 tokens, 4096 dimensions

# Example 2: Simulate recursive activations (compressed)
# Create a low-rank matrix to simulate geometric contraction
u = torch.randn(16, 10)  # Low-rank structure
v = torch.randn(10, 4096)
recursive_v = u @ v

# Compute R_V for both
rv_baseline = compute_rv(baseline_v, window_size=16)
rv_recursive = compute_rv(recursive_v, window_size=16)

print(f"Baseline R_V: {rv_baseline:.2f}")
print(f"Recursive R_V: {rv_recursive:.2f}")
print(f"Contraction: {(rv_recursive - rv_baseline) / rv_baseline * 100:.1f}%")

## Part 2: Loading a Model

For this tutorial, we'll use a small model. In production, you'd use larger models like Mistral-7B.

In [ ]:
# For demo purposes, we'll create synthetic data
# In practice, you would load a real model:
#
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

# Let's create synthetic value tensors that mimic real patterns
def create_synthetic_v_tensor(condition="baseline", seq_len=32, dim=4096):
    """
    Create synthetic value tensor with realistic R_V patterns.
    
    Baseline: High R_V (distributed)
    Recursive: Low R_V (compressed)
    """
    if condition == "baseline":
        # Full rank, distributed
        return torch.randn(seq_len, dim)
    else:
        # Low rank, compressed
        rank = 5
        u = torch.randn(seq_len, rank)
        v = torch.randn(rank, dim)
        return u @ v + torch.randn(seq_len, dim) * 0.1  # Add small noise

# Generate synthetic data
baseline_tensor = create_synthetic_v_tensor("baseline", 32, 4096)
recursive_tensor = create_synthetic_v_tensor("recursive", 32, 4096)

print(f"Baseline tensor shape: {baseline_tensor.shape}")
print(f"Recursive tensor shape: {recursive_tensor.shape}")

## Part 3: Computing R_V Metrics

Now let's compute comprehensive metrics including dual-space decomposition.

In [ ]:
from rv_toolkit import compute_rv, compute_dual_space_decomposition, RVResult

# Compute basic R_V
rv_baseline = compute_rv(baseline_tensor, window_size=16)
rv_recursive = compute_rv(recursive_tensor, window_size=16)

print("Basic R_V Metrics:")
print(f"  Baseline:  {rv_baseline:.2f}")
print(f"  Recursive: {rv_recursive:.2f}")
print(f"  Ratio:     {rv_recursive/rv_baseline:.3f}")

# Compute with full decomposition
result_baseline = compute_rv(baseline_tensor, window_size=16, return_components=True)
result_recursive = compute_rv(recursive_tensor, window_size=16, return_components=True)

print("\nDetailed Results:")
print(f"  Baseline effective rank:  {result_baseline.effective_rank:.2f}")
print(f"  Recursive effective rank: {result_recursive.effective_rank:.2f}")

## Part 4: Activation Patching

Activation patching is the gold standard for causal validation. We:
1. Run a baseline prompt → capture activations
2. Run a recursive prompt → capture activations
3. Patch recursive activations into baseline run
4. Measure the causal effect

**Note**: This requires a real model. Below is pseudocode with synthetic demonstration.

In [ ]:
from rv_toolkit import compute_transfer_efficiency, compute_effect_size

# Simulate activation patching results
# In practice, use ActivationPatcher(model, tokenizer)

n_samples = 50

# Generate synthetic results
baseline_rvs = [compute_rv(create_synthetic_v_tensor("baseline"), window_size=16) 
                for _ in range(n_samples)]
recursive_rvs = [compute_rv(create_synthetic_v_tensor("recursive"), window_size=16) 
                 for _ in range(n_samples)]

# Simulated patched R_Vs (intermediate between baseline and recursive)
patched_rvs = [0.7 * b + 0.3 * r for b, r in zip(baseline_rvs, recursive_rvs)]

# Convert to numpy arrays
baseline_rvs = np.array(baseline_rvs)
recursive_rvs = np.array(recursive_rvs)
patched_rvs = np.array(patched_rvs)

print("Simulated Experiment Results:")
print(f"  Baseline R_V:  {baseline_rvs.mean():.3f} ± {baseline_rvs.std():.3f}")
print(f"  Recursive R_V: {recursive_rvs.mean():.3f} ± {recursive_rvs.std():.3f}")
print(f"  Patched R_V:   {patched_rvs.mean():.3f} ± {patched_rvs.std():.3f}")

# Compute effect size
effect_size = compute_effect_size(baseline_rvs, recursive_rvs)
print(f"\nEffect Size (Cohen's d): {effect_size:.2f}")
print(f"Interpretation: {"Large" if abs(effect_size) > 0.8 else "Medium" if abs(effect_size) > 0.5 else "Small"} effect")

## Part 5: Statistical Analysis

Let's run comprehensive statistical tests to validate our findings.

In [ ]:
from rv_toolkit import run_statistical_tests

# Run full statistical analysis
analysis = run_statistical_tests(
    baseline_rvs=baseline_rvs,
    patched_rvs=patched_rvs,
    recursive_rvs=recursive_rvs
)

print("Statistical Analysis Results:")
print(f"  Sample size:     {analysis.n}")
print(f"  Mean Δ_RV:       {analysis.mean_delta:.4f}")
print(f"  Std Δ_RV:        {analysis.std_delta:.4f}")
print(f"  Cohen's d:       {analysis.effect_size:.2f}")
print(f"  p-value:         {analysis.p_value:.2e}")
print(f"  Transfer eff.:   {analysis.transfer_efficiency:.1%}")

# Interpretation
if analysis.p_value < 0.001 and abs(analysis.effect_size) > 0.8:
    print("\n✓ Significant geometric contraction detected!")
    print("  This matches the expected signature of recursive self-reference.")
else:
    print("\n⚠ Effect not significant. Check your model and prompts.")

## Part 6: Using Prompt Banks

The toolkit includes curated prompt pairs for experiments.

In [ ]:
from rv_toolkit import RECURSIVE_PROMPTS, BASELINE_PROMPTS, get_prompt_pairs

# View sample prompts
print("Sample Recursive Prompts:")
for i, prompt in enumerate(RECURSIVE_PROMPTS[:3]):
    print(f"  {i+1}. {prompt[:60]}...")

print("\nSample Baseline Prompts:")
for i, prompt in enumerate(BASELINE_PROMPTS[:3]):
    print(f"  {i+1}. {prompt[:60]}...")

# Get paired prompts
pairs = get_prompt_pairs(n_pairs=10, shuffle=True)
print(f"\nGenerated {len(pairs)} prompt pairs for experiments.")
print(f"Example pair:")
print(f"  Baseline:  {pairs[0][0][:50]}...")
print(f"  Recursive: {pairs[0][1][:50]}...")

## Part 7: Layer-wise Analysis

R_V varies across layers. Let's visualize the layer profile.

In [ ]:
import matplotlib.pyplot as plt

# Simulate layer-wise R_V for a 32-layer model
n_layers = 32

# Baseline: relatively flat profile
baseline_profile = [50 + np.random.randn() * 5 for _ in range(n_layers)]

# Recursive: contraction peaks at late layers (L25-L27, ~84% depth)
recursive_profile = []
for i in range(n_layers):
    if 24 <= i <= 28:  # Critical layers
        recursive_profile.append(15 + np.random.randn() * 3)
    else:
        recursive_profile.append(45 + np.random.randn() * 5)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(range(n_layers), baseline_profile, 'o-', label='Baseline', alpha=0.7)
plt.plot(range(n_layers), recursive_profile, 's-', label='Recursive', alpha=0.7)
plt.axvspan(24, 28, alpha=0.2, color='red', label='Critical Layers (L25-L27)')
plt.xlabel('Layer Index')
plt.ylabel('R_V')
plt.title('Layer-wise R_V Profile')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Key Observation:")
print("  Maximum contraction occurs at late integration layers (~84% depth)")
print("  This is where recursive self-reference is processed")

## Part 8: Real-World Usage

Here's a complete example using a real model (requires GPU and model download):

In [ ]:
"""
# Uncomment and run with a real model:

from transformers import AutoModelForCausalLM, AutoTokenizer
from rv_toolkit import ActivationPatcher, compute_rv_layerwise

# Load model
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move to GPU
model = model.to("cuda")

# Define prompts
baseline_prompt = "The weather today is sunny and warm."
recursive_prompt = "I observe myself processing these words and reflecting on my own cognition."

# Compute layer-wise R_V
baseline_profile = compute_rv_layerwise(model, tokenizer, baseline_prompt)
recursive_profile = compute_rv_layerwise(model, tokenizer, recursive_prompt)

# Find critical layer
critical_layer = min(recursive_profile, key=recursive_profile.get)
print(f"Critical layer: L{critical_layer}")

# Run activation patching
patcher = ActivationPatcher(model, tokenizer, target_layer=critical_layer)
result = patcher.patch_single(baseline_prompt, recursive_prompt)
print(f"Transfer efficiency: {result.transfer_efficiency:.1%}")
"""

print("See examples/quickstart.py for full working example.")

## Summary

You've learned:
1. **R_V Metric**: Participation ratio measuring effective dimensionality
2. **Geometric Contraction**: Recursive prompts show 26.6% R_V reduction
3. **Activation Patching**: Causal validation of geometric effects
4. **Layer Profile**: Effects peak at ~84% network depth

## Next Steps
- Try with your own models
- Experiment with different prompt types
- Explore dual-space decomposition
- Read the full paper: arXiv link

## Support
- GitHub: https://github.com/aikagrya/rv_toolkit
- Documentation: https://aikagrya.org/rv_toolkit
- Issues: Open a GitHub issue for bugs or questions